<a href="https://colab.research.google.com/github/weammoghazy/MLEmailSpamFilter/blob/master/RecommendationEngine_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install apache-beam

In [0]:
import apache_beam as beam
import tensorflow as tf
from apache_beam.options.pipeline_options import PipelineOptions

import re
import pandas as pd
import numpy as np

from google.colab import files

In [0]:
# create an empty pipeline
options = PipelineOptions()
p = beam.Pipeline(options=options)

In [0]:
uploaded = files.upload()

In [0]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

In [0]:
# read a csv file
dff = pd.read_csv('data_users__doc_0_611.csv', header=0)

# extract the header row
file_header = dff.columns.values

# convert the ndarray into a list (on which we can run index() method)
file_header = list(np.array(file_header))

In [0]:
# takes a header row, searches for skillsArray, and returns an array of skills column indices 
def skills_array(header):
    i = 0
    skills_indices = []
    while (True):
        skill = 'skillsArray[' + str(i) + ']'
        if skill not in header:
            break
        skills_indices.append(header.index(skill))
        i += 1
    return skills_indices

In [0]:
inputs_pattern = 'data_users__doc_0_611.csv'
outputs_prefix = './output.csv'
      
class Split(beam.DoFn):
  my_list = list()
  
  def __init__(self, indxArr):
    super(Split, self).__init__()
    self.indxArr = indxArr
    
  def process(self, element):
    temp_list = []
    my_list = element.split(',')
    for i in range (0,len(self.indxArr)):
      temp_list.append(my_list[self.indxArr[i]])
    return [temp_list]  

with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'split' >> beam.ParDo(Split(skills_array(file_header)))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

#run('head -n 20 {} -00000-of-*'.format(outputs_prefix))